In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import sys
from sklearn import metrics
from joblib import dump, load
from pathlib import Path

from art.attacks.evasion import FastGradientMethod, ZooAttack
from art.estimators.classification import SklearnClassifier

from art.estimators.classification.scikitlearn import ScikitlearnRandomForestClassifier

In [2]:
features = ['srcPort', 'destPort',
       'bytes_out', 'num_pkts_out', 'bytes_in', 'num_pkts_in', 'f_ipt_mean',
       'f_ipt_std', 'f_ipt_var', 'f_ipt_skew', 'f_ipt_kurtosis', 'f_b_mean',
       'f_b_std', 'f_b_var', 'f_b_skew', 'f_b_kurtosis', 'duration', 'pr',
       'domainId']
label = 'deviceId'

fieldTypes = {'deviceId': 'int16', 
        #'time_start': 'datetime64[ns]', 'time_end': 'datetime64[ns]', 
        'srcPort': 'int32', 'destPort': 'int32',
        'bytes_out': 'int32', 'num_pkts_out': 'int32',
        'bytes_in': 'int32', 'num_pkts_in': 'int32',
        'f_ipt_mean': 'float64', 'f_ipt_std': 'float64',
        'f_ipt_var': 'float64', 'f_ipt_skew': 'float64',
        'f_ipt_kurtosis': 'float64',
        'f_b_mean': 'float64', 'f_b_std': 'float64',
        'f_b_var': 'float64', 'f_b_skew': 'float64',
        'f_b_kurtosis': 'float64',
        'duration': 'float64', 'pr': 'int8',
        'expire_type': 'object',
        'domain': 'object',
        'domain2': 'object',
        'domainId': 'int16'}

In [ ]:
df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])
df.fillna(0, inplace=True)

weeks = [list(range(44,53)), list(range(1,10)), list(range(10,19))]


In [7]:

#trainCon = df['time'] >= pd.Timestamp('2020-03-01')
#trainCon = df['time'] < pd.Timestamp('2020-01-01')
#trainCon = ((df['time'] >= pd.Timestamp('2020-01-01')) & (df['time'] < pd.Timestamp('2020-03-01')))
trainCon = df['time_start'].dt.isocalendar().week.isin(weeks[0])
#trainCon = df['time'].dt.week.isin([44,45])


#testCon = df['time'] > pd.Timestamp('2020-02-29')
#testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1W').tolist()
testCons = pd.date_range('2019-12-01', '2020-05-01', freq = '1W').tolist()
#testCons = pd.date_range('2019-11-01', '2019-12-01', freq = '1W').tolist()

#X = dfTrain.iloc[:, 2:5]
#y = dfTrain.iloc[:, 1]
X = df[trainCon][features]
y = df[trainCon][label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=4700063, stratify=y)

In [9]:
rfc = RandomForestClassifier(n_jobs=18)
modelFile = Path(f"model/rfc_justvalues_0")
if modelFile.is_file():
    print(f"loading model {modelFile}")
    rfc = load(modelFile)

loading model model/rfc_justvalues_0


In [41]:
advClassifier = SklearnClassifier(rfc)
zoo = ZooAttack(classifier=advClassifier, confidence=0.0, targeted=False, learning_rate=1, max_iter=2,
                    binary_search_steps=2, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=1, batch_size=1, variable_h=0.2)


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
for date in testCons:
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[features]
    y = testDF[label]
    
    if len(X) == 0: 
        continue
    
    #y_pred = clf.predict(X)
    y_rfc = rfc.predict(X)
    #print("rfc predict")
    #y_svc = svc.predict(X)
    #print("svc predict")
    #y_dtc = dtc.predict(X)
    #print("dtc predict")
    #y_knn = knn.predict(X)
    #print("knn predict")
    #y_mv = mv.predict(X)
    #print("mv predict")
    
    print("Benign test")
    print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))
    print(classification_report(y,y_rfc))#, labels=devs))
            #metrics.f1_score(y, y_svc, average='micro'),
            #metrics.f1_score(y, y_dtc, average='micro'),
            #metrics.f1_score(y, y_knn, average='micro'),
            #metrics.f1_score(y, y_mv, average='micro')
            #)
    #print(f"Accuracy {date.date}:", metrics.accuracy_score(y, y_pred))

    # Step 6: Generate adversarial test examples
    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


Benign test
48 0.9299325227261479


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.96      0.96      0.96     14549
           3       0.99      0.86      0.92     59799
           5       1.00      0.99      0.99      1539
           6       0.86      0.85      0.85    173354
           7       0.97      0.97      0.97     11337
           8       0.96      0.96      0.96     30712
           9       0.97      0.97      0.97     14478
          10       0.99      1.00      1.00     28244
          11       1.00      1.00      1.00     16021
          12       0.94      0.58      0.72     19080
          13       0.81      1.00      0.90    292825
          14       1.00      1.00      1.00      9396
          15       0.66      0.59      0.62     27064
          16       0.85      0.80      0.82    155640
          17       0.99      0.99      0.99     18584
          18       0.96      0.96      0.96     19210
          19       0.95    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


...

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


Benign test
9 0.8181996622802308


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.90      0.90      0.90     15172
           3       0.52      0.74      0.61     10733
           5       0.99      0.99      0.99      1446
           6       0.53      0.64      0.58    341646
           7       0.94      0.91      0.93     12680
           8       0.37      0.39      0.38     16596
           9       0.00      0.00      0.00         0
          10       1.00      0.99      0.99     65005
          11       0.91      0.97      0.94     12337
          12       0.80      0.47      0.59     18723
          13       0.96      0.97      0.96    252366
          14       0.00      0.00      0.00         0
          15       0.58      0.64      0.61     18493
          16       0.34      0.24      0.28    185774
          17       0.98      0.97      0.98     18032
          18       0.91      0.91      0.91     19196
          19       0.65    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [45]:

for date in testCons:
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[features]
    y = testDF[label]
    
    if len(X) == 0: 
        continue
    x_test_adv = zoo.generate(x=X.values)
   
    np.savetxt(f"RFzoo-{date.week}.csv", x_test_adv, delimiter=",")

   # Step 7: Evaluate the ART classifier on adversarial test examples
    y_rfc = advClassifier.predict(x_test_adv)
    print("ZOO test")
    print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))
  


ZOO:   0%|          | 2681/2788198 [06:28<100:40:18,  7.69it/s]Exception ignored in: <function _ConnectionBase.__del__ at 0x14c114fe6c10>
Traceback (most recent call last):
  File "/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/multiprocessing/connection.py", line 130, in __del__
    def __del__(self):
KeyboardInterrupt: 
ZOO:   0%|          | 2688/2788198 [06:29<112:06:40,  6.90it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/tmp/pbs.8239880.pbs/ipykernel_781602/1322874622.py", line 11, in <module>
    x_test_adv = zoo.generate(x=X.values)
  File "/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/attacks/evasion/zoo.py", line 236, in generate
    res = self._generate_batch(x_batch, y_batch)
  File "/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/attacks/evasion/zoo.py", line 280, in _generate_batch
    best_dist, best_label, best_attack = self._generate_bss(x_batch, y_batch, c_current)
  File "/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/attacks/evasion/zoo.py", line 408, in _generate_bss
    preds, l2dist, loss = self._loss(x_orig, x_adv, y_batch, c_batch)
  

In [ ]:
for date in testCons:
   
    # Step 7: Evaluate the ART classifier on adversarial test examples
    y_rfc = advClassifier.predict(x_test_adv)
  
    print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
    print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))